# Sem Tech Project Gruppe 1

## Define Company Object Class

In [1]:
class CompanyObject:
    def __init__(self, list_Languages, list_Lables, list_Aliases, isSubsidiaryOf="None",
                 TwitterUsernames="", TwitterIDs=0, compID=0):
        
        self.compID=compID
        self.list_Languages=list_Languages
        self.list_Lables=list_Lables
        self.list_Aliases=list_Aliases
        self.isSubsidiaryOf=isSubsidiaryOf
        self.TwitterUsernames=TwitterUsernames
        self.TwitterIDs=TwitterIDs
    
    def __str__(self):
        s="ID: "+str(self.compID)+" with "+str(len(self.list_Languages))+" languages and TwitterUsernames: "+str(self.TwitterUsernames)
        if(self.isSubsidiaryOf != "null"):
            s+=" with Subsidiary: "+str(self.isSubsidiaryOf.compID)
        return s

# Crawl HTML Code

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


### First Tests of HTML Extraction

def AddSocialMediaToCompany(comp):
    response = requests.get(url=("https://www.wikidata.org/wiki/"+comp.compID))
    soup = BeautifulSoup(response.content, 'html.parser')
    http = soup.find_all('a', {'href':re.compile('https://twitter.com')})
    nameList=[]
    idList=[]
    for i in range(0, len(http)):
        url=http[i]['href'].rsplit('/', 1)[-1]
        if(url.isdigit()):
            idList.append(url)
        else:
            nameList.append(url)
    comp.TwitterUsernames=nameList
    comp.TwitterIDs=idList
    
    
response = requests.get(
	url="https://www.wikidata.org/wiki/Q20716",
)
soup = BeautifulSoup(response.content, 'html.parser')


twittername = soup.find_all("div", id=re.compile("Q20716\$"))
                            
http = soup.find_all('a', {'href':re.compile('https://twitter.com')})

#print(http)
#url = "https://twitter.com/i/user/2381578122"
#print(url.rsplit('/', 1)[-1])
#twitterNameList = [http[i] for i in range(0, len(http), 2)]

nameList=[]
idList=[]

#userid = [id for in http]
#print(http[0]['href'])

for i in range(0, len(http)):
    url=http[i]['href'].rsplit('/', 1)[-1]
    if(url.isdigit()):
        idList.append(url)
    else:
        nameList.append(url)
#print(idList,nameList)



# Crawl Json

In [3]:
import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

def isNaN(num):
    return num!= num

def getDaughterID(comp):
    return comp['besteht_aus']['value'].rsplit('/', 1)[-1]

def crawl(compID, compMotherObject="null"):
    response = requests.get(
        url=("https://www.wikidata.org/wiki/Special:EntityData/"+compID+".json"))
    data = json.loads(response.text)
    df = pd.DataFrame(data['entities'][compID]['labels'])
    
    languages=df.iloc[0]
    labels=df.iloc[1]
    
    df = pd.DataFrame(data['entities'][compID])
    
    temp = [i for i in df['aliases'] if not (isNaN(i))]
    
    aliases=[]

    for i in temp:
        for a in i:
            aliases.append(a)
    
    # Example Accesses
    #print(new,"ENDE")
    #print(new[0]['language'])
    #print(new[0]['value'])

    
### TODO: Get further Data from HTML
    
     
    compObject = CompanyObject(languages, labels, aliases,compID=compID,
                               TwitterUsernames="Unknown", isSubsidiaryOf=compMotherObject
                             )

    AddSocialMediaToCompany(compObject)
    print(compObject)
    
    return compObject

#crawl('Q20716')

# Länge der Liste
#print(len(results["results"]["bindings"]))

#for result in results["results"]["bindings"][:10]:
#    pass

## Get whole Company List

In [5]:
# pip install sparqlwrapper
# pip install pandas
# https://rdflib.github.io/sparqlwrapper/

import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

### Die aktuelle Query, welche zu jeder Company auch noch die SubCompanies liefert

query="""SELECT ?company ?besteht_aus WHERE {
  ?company wdt:P31 wd:Q4830453.
  OPTIONAL { ?company wdt:P1830 ?besteht_aus. }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# Länge der Liste

size=len(results["results"]["bindings"])
#print(size)

#print(results["results"]["bindings"][:10])

#liste = results["results"]["bindings"][:10]

#print(liste[0])

wholeCompList=[]
#for i in range(0,size)
#for i in range(0,100):
i=0








### Change number of Iterations here:

while(i<11):
    print("----------------",i,"------------------------")
    id=results["results"]["bindings"][i]['company']['value'].rsplit('/', 1)[-1]
    

    
    list_tovisit=[]
    list_tovisit.append(results["results"]["bindings"][i]['besteht_aus']['value'].rsplit('/', 1)[-1])
    
    for j in range(i+1,size):
        nextElem=results["results"]["bindings"][j]
        if (nextElem['company']['value'].rsplit('/', 1)[-1] == id):
            list_tovisit.append(getDaughterID(nextElem))
        else:
            i=j
            break;
            
    company = crawl(id)
    wholeCompList.append(company)
    for e in list_tovisit:
        print("................................")
        wholeCompList.append(crawl(e,company))
       # time.sleep(2)
    
for comp in wholeCompList:
    ### Do Whatever you want ;)
    pass
print("We Are Done and Happy")

---------------- 0 ------------------------
ID: Q66 with 109 languages and TwitterUsernames: ['Boeing']
................................
ID: Q8793 with 47 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q374451 with 23 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q570930 with 17 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q795324 with 29 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q890212 with 23 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q961568 with 11 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q1236833 with 37 languages and TwitterUsernames: ['ulalaunch'] with Subsidiary: Q66
................................
ID: Q1541150 with 12 languages and TwitterUsernames: [] with Subsidiary: Q66
.............

## -------------------------------------------------------------

# Setting up the ontology

In [ ]:
from owlready2 import *

#load the basic structure of the ontology that was created in webprotege
onto = get_ontology("file://names_onto.owl").load()

classes = []

#extract the classes so we can create instances
for cl in onto.classes():
    classes.append(cl)



# Basic functions to create instances in order to fill our ontology

In [ ]:
#TODO add attributes for instances when needed. Run for all aquired data.

def get_Company_Instance(name):
    cl = [c for c in classes if c.name == "Company"]
    return cl[0](name)

def get_Alias_Instance(name):
    cl = [c for c in classes if c.name == "Alias"]
    return cl[0](name)

def get_Language_Instance(name):
    cl = [c for c in classes if c.name == "Language"]
    return cl[0](name)

def get_Dialect_Instance(name):
    cl = [c for c in classes if c.name == "Dialect"]
    return cl[0](name)

def get_Name_Instance(name):
    cl = [c for c in classes if c.name == "Name"]
    return cl[0](name)

def get_TikTok_Instance(name):
    cl = [c for c in classes if c.name == "TikTik_Username"]
    return cl[0](name)

def get_Twitter_Instance(name):
    cl = [c for c in classes if c.name == "Twitter_Username"]
    return cl[0](name)
    
def get_Youtube_Instance(name):
    cl = [c for c in classes if c.name == "Youtube_Username"]
    return cl[0](name)

def get_SubCompany_Instance(name):
    cl = [c for c in classes if c.name == "Subsidiary_company"]
    return cl[0](name)



In [ ]:
#never run this again unless absolutely necessary, takes a while..

values = []
owned_by = []
companies = []

for company in results["results"]["bindings"]:
    entity_number = company["company"]["value"][31:] 
    try: 
        sub_number = company["besteht_aus"]["value"][31:]  
        owned_by.append({"owner": entity_number, "owned": sub_number})
    except KeyError:
        pass
    values.append(entity_number)

for value in values:
    company = get_Company_Instance(value)
    companies.append(company)


for pair in owned_by:
    if pair["owner"] in values and pair["owned"] in values:
        owner = [c for c in companies if c.name == pair["owner"]][0]
        owned = [c for c in companies if c.name == pair["owned"]][0]
        owner.is_parent_company_of.append(owned)



# Example on how to link created instances via properties

In [ ]:

"""
alias = get_Alias_Instance("Alias")

Name = get_Name_Instance("Name")

alias.belongs_to.append(Name)
"""


# Some sample querries running on our ontology

In [ ]:
print("Number of classes:", list(default_world.sparql("""
           SELECT (COUNT(?x) AS ?nb)
           { ?x a owl:Class . }
    """)))

print("One example class", list(default_world.sparql("""
           SELECT ?x
           { ?x rdfs:label "Alias" . }
    """)))

print("All classes with their sub-classes:", list((default_world.sparql("""
           SELECT ?x ?y
           { ?x a owl:Class . 
             ?x rdfs:subClassOf* ?y }
    """))))


# Using sync_reasoner to find and fix inconsistencies 

In [ ]:
#automatically finds inconsistencies and reclassifies individuals and superclasses
#especially helpful when circular dependencies are present 
sync_reasoner()


#optional: use diffrent reasoner, no differences spotted yet
#sync_reasoner_pellet() 


#optional: also taking object property values into acount
#sync_reasoner(infer_property_values = True)

# Saving changes

In [ ]:
# weird: owlready2 cannot overwrite a file passed into the function via the first parameter. It can only create a new files...

onto.save("filled_names_onto4", "rdfxml")